In [33]:
import requests
import pandas as pd
from multiprocessing.pool import ThreadPool


In [100]:
def get_openpagerank(domains):
    url = "https://openpagerank.com/api/v1.0/getPageRank"
    headers = {
        "API-OPR": "oksws8wgs04ws8k480scs0ok48k00skcw8swgso0"
    }
    params = {
        "domains[]": domains  # Pass domains as a list
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an error for bad responses
        return domains, response.json()  # Return the JSON response
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")




In [72]:
unique_domains = pd.read_csv("sample_domain_list.csv",header=None)
unique_domains = list(unique_domains[0])


In [74]:
list_of_domainList = [unique_domains[i:i + 100] for i in range(0, len(unique_domains), 100)]


In [111]:
with ThreadPool(10) as pool:
    results = pool.map(get_openpagerank, list_of_domainList)
    pool.close()
    pool.join()
    results_df = pd.DataFrame(data = results)
    

In [112]:
combined_domain_df = pd.DataFrame(columns=['status_code', 'error', 'page_rank_integer', 'page_rank_decimal',
       'rank', 'domain'])
for idx in results_df.index:
    temp_df = pd.DataFrame(results_df.loc[idx,1]["response"])
    website_df = pd.DataFrame(results_df.loc[idx,0])
    website_df.columns = ["website"]
    response_df = pd.concat([temp_df,website_df],axis=1)
    combined_domain_df = pd.concat([combined_domain_df,response_df])
   

In [113]:
combined_domain_df

,status_code,error,page_rank_integer,page_rank_decimal,rank,domain,website
0,200,,5,4.91,222102,rightwingnews.com,rightwingnews.com
1,200,,5,4.89,319092,swishappeal.com,www.swishappeal.com
2,200,,5,5.11,85262,tsln.com,www.tsln.com
3,200,,4,4.48,1202524,1035kissfmboise.com,1035kissfmboise.com
4,200,,3,3.41,4792714,myradioplace.com,myradioplace.com
...,...,...,...,...,...,...,...
99,200,,5,4.89,328137,piquenewsmagazine.com,www.piquenewsmagazine.com
0,200,,4,3.77,3077733,adventuremomblog.com,adventuremomblog.com
1,200,,5,4.90,246340,980waav.com,www.980waav.com
2,200,,3,2.94,10912385,maqnews.com,www.maqnews.com


In [114]:
combined_domain_df.to_csv("sample_page_rank_domains.csv",index=False)